# Software HW 4

_Due 2/7/2024_

For submission instructions, see the last section!

Lets build on the classes objects that we looked at during class. The goal is to write some code that will calculate the missing momenta of a collision. We'll need to build a few new tools in order to do that.

First, in the [ATLAS](https://atlas.cern/) experiment two protons are collided - both are going the same speed when they hit - so the lab frame is the same as the center of mass frame. Since momentum conservation holds as well in particle physics as it does anywhere else, since the initial momenta is zero the final momenta must also be zero (see note below). As a result, summing the total momenta of all measured particles is a very useful way to see if we've missed anything! This missed thing could be a neutrino (which does not interact with the detector and so carries away some energy), or it could be dark matter! As a result, missing momentum is a very hot topic!

_NOTE_: $\vec{p} = 0$ isn't 100% true for two reasons. The main one is that at the energies that ATLAS runs at the protons aren't what interact - it is the quarks inside the protons. And the quarks are moving relative to the proton's center-of-mass. This is only an issue along the boost direction (beam direction). As a result it is only $p_x$ and $p_y$ that we rely on being zero after the collision! On the other hand, $\vec{p} = 0$ is true in a lepton collider (like the LEP collider at CERN or AMY in Japan). For this HW, we'll consider all three dimensions.

First, make sure the `particle` class is properly installed:

In [ ]:
!pip install particle

In [2]:
from particle import Particle

And now we need the definition of `AParticle` again. Just repeat what we did during class. Don't forget to add that `__repr__` method we added on the fly - it will make understanding what is going on below a lot easier!

## Generate An Event

First we need to generate an event. Lets make event have 1000 `AParticle`'s in it, and $p_x, p_y, p_z$ will be random between -100 and 100.

To do this we need to learn about python's random number generator. The ones we use come from the [`random` module](https://docs.python.org/3/library/random.html), which is part of a python distribution. If you are doing anything "big" you'll use other random number generators, like the ones in the [`numpy` module](https://numpy.org/doc/stable/reference/random/generator.html). There are several things to be aware when using a random number generator:

* If you are going to do something requiring a large number of random number generators it pays to find a good generator. The quality of the generators is all over the map. Any that have a _pattern_ will mean you will likely explore some parts of your phase space than others - thus leading to a bias in your results.
* The second thing is that random number generators are... random. This means if you run your result twice you won't get the same answer. This can be particularly vexing when you are trying to track down a bug. To this end, most scientific codes carefully track the [random number _seed_](https://www.statisticshowto.com/random-seed-definition/#:~:text=A%20random%20seed%20is%20a%20starting%20point%20in,a%20computer%20system%E2%80%99s%20clock%20%28Henkemans%20%26%20Lee%2C%202001%29.).
* Even if we all use the same seed, there is no garuntee with most algoritms that we'll get the same sequence on all computers. I chose [42](https://www.dictionary.com/e/slang/42/) as my seed, and the sequence I get below may be different from what you see. But if you choose the same seed, and run on the same computer, you should always get the same sequence!

First we will set the seed to a particular value and run the generator. As you can see, when we reset the seed, we get the same answer:

In [ ]:
import random

random.seed(42)
print(random.uniform(-100, 100), random.uniform(-100, 100), random.uniform(-100, 100))
random.seed(42)
print(random.uniform(-100, 100), random.uniform(-100, 100), random.uniform(-100, 100))

As you can see - each sequence is random, but they are the same sequence since the seed is reset. So... ONLY RESET THE SEED ONCE AT THE START OF YOUR CODE - never in a loop. ;-)

Ok - so now that we have a reproducible random number generator, lets generate a list with 1000 tracks in it. First, lets write a function that will generate a single, random track:

* It should return a `AParticle`
* It should be of type `particle` (see function below)
* The $p_x, p_y, p_z$ should be in the range -100 to 100 (we'll say units are MeV).

Here is the function skeleton. Replace the `0` in the return statement, and/or add any lines of code before the `return` statement you need:

In [ ]:
def generate_particle(particle):
    '''Generates a single particle with a random momentum between -100 and 100 on each axis.

    Args:
        particle (Particle): The particle type of the track we will generate.

    Returns:
        AParticle: The generated particle.
    '''
    return 0

And if you did everything right:

In [ ]:
ele = Particle.from_evtgen_name('e-')
print(generate_particle(ele))

It should look something like `(e- 61.89, -98.70, 61.16)` - the exact formatting will depend on your `__repr__` you used above (and remember not all random number generators are the same!). You can play with it and format specifiers if you want to get things looking better than they do by default (just search for _python format specifiers_).

Ok - we can generate a single particle. Next, lets generate a list of 100 electron particles. There are two ways to do this - one is with a `for` loop, and the other is with a `for` statement (as discussed in class). Lets try the for statement. The function that does this should have a single line that returns the list:

In [ ]:
def generate_tracks(particle, n):
    '''Generates a list of particles with a random momentum between -100 and 100 on each axis.

    Args:
        particle (Particle): The particle type of the track we will generate.
        n (int): The number of tracks to generate.

    Returns:
        list: A list of AParticle objects.
    '''
    return 0

And the following `assert` statements should pass if you got the above right:

In [ ]:
track_list = generate_tracks(ele, 100)
assert len(track_list) == 100
# This next line checks to see if all the elements in the list are AParticle objects
assert all([isinstance(track, AParticle) for track in track_list])

## The magnitude of the momentum

Lets add to the `AParticle` class a method that calculates the magnitude of the momentum (don't forget the `sqrt`). Look at python's `math` module for information on things like `sqrt` and how to use it.

If we were developing this code normally, and not in a software HW, we'd put the `AParticle` class definition in a separate file - as it has become a useful utility class. And then we'd re-start the notebook. We'll do something a bit more fast-and-loose this time, though in the future we'll move to that model.

In the cell below, reproduce the `AParticle` class definition from above, and add a method called `mag`:

```python
    def mag(self):
        '''Return the magnitude of the momentum of this track.
        '''
        return 0
```

And replace the `0` in the `return` statement with the calculation of the magnitude of momentum.

And this next cell tests that the code works:

In [ ]:
assert AParticle(ele, 1, 2, 3).mag() == math.sqrt(1 + 2*2 + 3*3)

Lets made a plot of the magnitude of 1000 generated events! First, lets generate 1000 tracks, and then transform the generated events into 1000 magnitudes. In this next cell, create a variable called `track_list`, by calling `generate_tracks`. Then use list comprehension to transform the `track_list` into `mag_list`, a sequence of magnitude numbers.

And this next test just to make sure we got what we needed.

In [ ]:
assert all([isinstance(mag, float) for mag in mag_list])

And lets make a plot! We want to how often each magnitude appears in the list - this is a frequency type plot. For that, we'll use a histogram. We'll use the `matplotlib` library for this. Your book has a chapter on `matplotlib` and plotting, but since we've not covered that yet, I'll provide the code as a template for later user here:

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.hist(mag_list, bins=100, range=(0, 200))
plt.xlabel('Momentum Magnitude [MeV/c]')
plt.ylabel('Number of Tracks')
plt.title('Momentum Magnitude of Electron Tracks')
plt.show()

A new things to note - for any good plot in science!!

* Axis labels
* Units where appropriate
* Title

It might seem crazy to put this stuff on for something quick and dirty, however, you'll often find yourself lifting your quick-and-dirty plots over to a lab group presentation - and standing there presenting a plot you made two days ago, one of 50, without units or axis titles means you'll forget...

The key line to make the plot is `plt.hist(mag_list, bins=100, range=(0, 200))`. First, know there are 1000 options you can use to alter the plot. The internet and [`stackoverflow`](https://stackoverflow.com) has lots of specific answers. The [manual page for `hist`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html) too.

But here:

* The first argument is a list of floating point numbers to be histogrammed
* `bins` is the number of bins to sort the data into. You can alter that and change the width of the histogram. Our histogram looks pretty ratty - lots of statistical jumps, so it has too many bins. Likely 50 or so would be sufficient. There is some theory behind this - you can determine the number of bins on the fly motivated by statistics. However, this makes overlaying plots very hard!
* `range` gives the low and high end of the x-axis. We could leave this off, and it will automatically be calculated. However, by choosing it yourself you get sensible numbers along the x-axis.

Your first plot in this class, awesome!

## Event Class and Missing Momentum

Final calculation step - lets calculate the missing momentum for an event. For this we want a function that will perform a vectorial sum of the momentum components of a list of tracks.

In [ ]:
def missing_momentum_mag(tracks):
    '''Calculates the magnitude of the missing momentum.'''
    return 0

And a test to make sure this works as expected

In [ ]:
t1 = AParticle(ele, 1, 1, 1)
assert missing_momentum_mag([t1]) == math.sqrt(1*1 + 1*1 + 1*1)
assert missing_momentum_mag([t1, t1]) == math.sqrt(2*2 + 2*2 + 2*2)
t2 = AParticle(ele, -1, -1, -1)
assert missing_momentum_mag([t1, t2]) == 0

In the next cell create an Event class.

* Its ctor should take a list of tracks as an argument
* It should have one method called `missing_momentum_mag` which uses the function you just wrote to calculate the event's missing momentum.
* Don't forget to document your code!!

And in the next cell a function called `generate_event` that takes as an argument the number of tracks to generate for a single event and returns an event.

And a quick test to see if this worked as expected:

In [ ]:
e = generate_event(ele, 1000)
assert len(e.tracks) == 1000
assert e.tracks[0].particle == ele

And lets make a plot of the missing momentum, finally! Copy the code from various cells above to:

* Generate 1000 events with 1000 tracks each
* Get the missing momentum magnitude for each event
* Plot the result. Leave the `range` keyword off to first get an idea of the plot, then add it back in. Use 50 bins for a better looking plot.

On my decent computer this took almost 5 seconds - for this, that is a long time. If we were serious about doing this, we'd use a datascience library, like `numpy` or the particle physics library, `awkward` to do this. Everything would have run much faster. Python isn't optimized for manipulating large lists of numbers and doing calculations on them - these other libraries are.

That said, we are using python 3.9 and 3.10. There have been some dramatic speed improvements in 3.11, and some others are set to come in 3.11.

## Bonus

If you wanted to do more, some ideas. These aren't graded and don't hand them in. But if you were looking for more challenges and wanted to extend the work above:

* Create a `vector` class that had `px, py, pz` as members, and could calculate the magnitude. Then re-write everything above to use this magnitude calculation. Note you now avoid having to re-write the sqrt of the sum of the squares. That cut-paste, btw, is a clear indication that might make you think of creating this `vector` class - and, indeed, is why most code like this is based on a vector-like class.
* Add `phi` and `theta` calculations to the `vector`, and then make similar plots for the event to show that the tracks distribution in those variables is uniform.
* Modify `generate_event` to take a list of `particle` types, and then randomly pick different particle types during the generation of the track list. The `random` python module has some items to help you with this. And you'll have to rewrite a few of the functions we have above. Finally, make a histogram of the # of each type of particle.
* Make a plot of missing momentum, but calculate the missing momentum for only the tracks that have a magnitude of momentum greater than 10 MeV.

# Submission

Fantastic! You are done!

To submit this HW:

1. Reset the kernel and run it again top to bottom. The cell numbers for code cells should all be in order!
2. Save this file
3. Use your web browser to print to PDF (or save as PDF) the open notebook.
4. Submit the PDF

You are done!